In [14]:
import sqlite3
import time
import shutil
from pathlib import Path

def merge_chroma_sqlite_with_copy(
    src_db_path: str,
    dst_db_path: str,
    tables: list[str],
    copy_suffix: str = "_copy",
    max_retries: int = 5,
    retry_delay: float = 5.0
):
    """
    소스와 대상 Chroma SQLite DB를 병합하되,
    대상 DB는 원본을 보존하고 대상_copy 파일을 생성하여 작업합니다.
    이후 대상_copy 파일에 src DB의 테이블 데이터를 이어붙입니다.
    'collections' 테이블은 스킵하며,
    INSERT OR IGNORE로 중복 엔트리를 무시합니다.

    Args:
        src_db_path: 소스 DB 파일 경로
        dst_db_path: 대상 DB 파일 경로
        tables: 병합할 테이블 이름 리스트
        copy_suffix: 대상 복사본 파일명에 붙일 접미사
        max_retries: 잠김 해제 재시도 횟수
        retry_delay: 재시도 대기 시간(초)
    """
    src = Path(src_db_path)
    dst = Path(dst_db_path)
    if not src.exists():
        raise FileNotFoundError(f"Source DB not found: {src}")
    if not dst.exists():
        raise FileNotFoundError(f"Destination DB not found: {dst}")

    # 1) 대상 DB 복사본 생성
    dst_copy = dst.with_name(dst.stem + copy_suffix + dst.suffix)
    shutil.copy2(dst, dst_copy)
    print(f"Created backup: {dst_copy}")

    # 2) 복사본에 병합 수행
    attempt = 0
    while True:
        try:
            conn = sqlite3.connect(str(dst_copy), timeout=30, uri=True)
            conn.execute("PRAGMA journal_mode=WAL;")
            conn.execute("PRAGMA busy_timeout=30000;")
            cur = conn.cursor()

            # 소스 DB 읽기 전용 ATTACH
            attach_str = f"file:{src}?mode=ro"
            cur.execute(f"ATTACH DATABASE '{attach_str}' AS src KEY ''; ")
            cur.execute("BEGIN;")

            for tbl in tables:
                if tbl == 'collections':
                    continue
                cur.execute(
                    f"INSERT OR IGNORE INTO {tbl} SELECT * FROM src.{tbl};"
                )
                print(f"Merged {cur.rowcount} rows into '{tbl}'")

            cur.execute("COMMIT;")
            cur.execute("DETACH DATABASE src;")
            conn.close()
            print(f"Merge complete on backup: {dst_copy}")
            break

        except sqlite3.OperationalError as e:
            if 'locked' in str(e).lower() and attempt < max_retries:
                attempt += 1
                print(f"DB locked, retry {attempt}/{max_retries} after {retry_delay}s...")
                time.sleep(retry_delay)
                continue
            raise

if __name__ == '__main__':
    src_db = r"C:/Users/Playdata/SKN13-4th-4Team/ai_influencer_agent/pilates_expert_blog_db/chroma.sqlite3"
    dst_db = r"C:/Users/Playdata/SKN13-4th-4Team/ai_influencer_agent/research_db/chroma.sqlite3"
    tables_to_merge = ['collections', 'segments', 'embeddings', 'embedding_metadata']

    merge_chroma_sqlite_with_copy(src_db, dst_db, tables_to_merge)

Created backup: C:\Users\Playdata\SKN13-4th-4Team\ai_influencer_agent\research_db\chroma_copy.sqlite3
Merged 2 rows into 'segments'
Merged 127437 rows into 'embeddings'
Merged 802822 rows into 'embedding_metadata'
Merge complete on backup: C:\Users\Playdata\SKN13-4th-4Team\ai_influencer_agent\research_db\chroma_copy.sqlite3
